In [19]:
import os
#from dotenv import load_dotenv
#load_dotenv()
from sqlalchemy.sql import func
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
from sqlalchemy import Column, Integer, String, Float
import numpy as np
import pandas as pd


#################################################
# Database Setup
#################################################
engines = create_engine("sqlite:///Datasets//Weather_3.db")

# Save reference to the table

conn = engines.connect()
Base = automap_base()
Base.prepare(engines, reflect=True)
session = Session(conn)
Base.classes.keys()

['State_Code', 'Weather_Raw']

In [19]:
# Performs database schema inspection
#insp = sqlalchemy.inspect(engines)
#print(insp.get_table_names())

['State_Code', 'Weather_Raw']


In [20]:
Wtr = Base.classes.Weather_Raw
St_Cd = Base.classes.State_Code
join = session.query( Wtr , St_Cd ).filter(Wtr.State == St_Cd.Code).statement

In [24]:
df = pd.read_sql_query(join, session.bind)
df

,State,County,Element,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Code,State_1
0,1,1,2,1895,44.0,38.2,55.5,64.1,70.6,78.3,80.4,80.4,79.0,61.4,54.4,45.3,1,Alabama
1,1,1,2,1896,44.3,49.0,54.0,69.3,76.8,78.0,81.7,83.1,77.9,64.7,58.0,47.3,1,Alabama
2,1,1,2,1897,43.7,52.3,61.3,63.0,70.0,82.4,82.4,79.6,76.6,67.4,54.9,48.2,1,Alabama
3,1,1,2,1898,50.1,46.8,60.1,59.6,75.0,81.5,80.8,79.2,76.2,62.1,50.2,44.2,1,Alabama
4,1,1,2,1899,44.6,41.5,56.6,62.3,76.7,81.0,81.0,81.5,74.3,66.6,55.7,45.3,1,Alabama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400631,50,290,2,2018,-2.2,6.1,15.1,25.6,43.6,54.6,60.0,51.0,43.6,30.4,11.7,2.7,50,Alaska
400632,50,290,2,2019,-1.8,9.4,25.0,29.2,47.0,59.3,62.0,51.7,43.9,27.5,10.7,-4.9,50,Alaska
400633,50,290,2,2020,-16.2,-7.4,7.9,27.3,47.7,55.9,55.9,55.1,42.1,24.8,5.9,3.1,50,Alaska
400634,50,290,2,2021,3.6,-9.0,4.4,22.9,44.3,56.3,58.0,51.1,39.1,25.7,-1.7,-0.4,50,Alaska
